In [32]:
from gurobipy import Model, GRB, quicksum
import pandas as pd

In [33]:
# Load the player data
players_df = pd.read_csv('/Users/mahinbindra/Downloads/BasketballPlayers.csv')

In [34]:
# Initialize the model
m = Model("team_selection")

In [35]:
# Decision variables: x[i] == 1 if player i is selected, 0 otherwise
x = m.addVars(len(players_df), vtype=GRB.BINARY, name="x")

In [36]:
# Position requirements based on predefined flags for guard and forward/center positions
guard_flags = [1 if pos in ['G', 'G/F'] else 0 for pos in players_df['Position']]
forward_center_flags = [1 if pos in ['F', 'C', 'F/C'] else 0 for pos in players_df['Position']]

In [37]:
# Apply position requirements
m.addConstr(quicksum(x[i] * guard_flags[i] for i in range(len(players_df))) >= 0.3 * 21, "min_guards")
m.addConstr(quicksum(x[i] * forward_center_flags[i] for i in range(len(players_df))) >= 0.4 * 21, "min_forwards_centers")

<gurobi.Constr *Awaiting Model Update*>

In [38]:
# Skill average requirement for each skill without division, optimized for clarity and efficiency
skills_columns = ['Ball Handling', 'Shooting', 'Rebounding', 'Defense', 'Athletic Ability', 'Toughness', 'Mental Acuity']
for skill in skills_columns:
    m.addConstr(quicksum(players_df[skill][i] * x[i] for i in range(len(players_df))) >= 2.05 * 21, f"skill_avg_{skill}")


In [39]:
# Specific invitations and exclusions, optimized for clarity
m.addConstr(quicksum(x[i] for i in range(19, 24)) + quicksum(x[i] for i in range(71, 78)) <= 1, "exclusion_rule_1")
m.addConstr(quicksum(x[i] for i in range(104, 114)) <= quicksum(x[i] for i in range(44, 49)) + quicksum(x[i] for i in range(64, 69)), "inclusion_rule_2")

<gurobi.Constr *Awaiting Model Update*>

In [40]:
# Diverse representation, ensuring at least one player from each specified group is selected
for k in range(0, 150, 10):  # Adjusted to cover the full range of players
    m.addConstr(quicksum(x[i] for i in range(k, min(k+10, len(players_df)))) >= 1, f"group_{k+1}-{k+10}")

In [41]:
# Total players constraint to ensure exactly 21 players are selected
m.addConstr(quicksum(x[i] for i in range(len(players_df))) == 21, "total_players")

<gurobi.Constr *Awaiting Model Update*>

In [42]:
# Objective function to maximize the total skill rating
m.setObjective(quicksum(quicksum(players_df[skill][i] * x[i] for i in range(len(players_df))) for skill in skills_columns), GRB.MAXIMIZE)

In [43]:
# Optimize model
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 27 rows, 150 columns and 1532 nonzeros
Model fingerprint: 0x44635937
Variable types: 0 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [9e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 0 rows and 1 columns
Presolve time: 0.01s
Presolved: 27 rows, 149 columns, 1184 nonzeros
Variable types: 0 continuous, 149 integer (149 binary)

Root relaxation: objective 3.600000e+02, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     360.0000000  360.00000  0.00%     -    0s

Explored 1 nodes (8 simplex iterati

In [44]:
# Print selected players
if m.status == GRB.Status.OPTIMAL:
    selected_players = [i for i in range(len(players_df)) if x[i].X > 0.5]
    print("Selected players:", selected_players)
else:
    print("No feasible solution found.")

Selected players: [6, 10, 25, 36, 46, 55, 67, 73, 89, 94, 103, 109, 110, 117, 127, 131, 132, 133, 140, 143, 147]


**Part A**
- Binary decision variables are used, where each variable xi epresents whether player i is selected (1) or not selected (0) for the training camp.

**Part B**
- The model must include as many decision variables as there are players in the dataset. If the provided solution includes players numbered up to 147, it implies the dataset has at least 147 players, requiring at least 147 decision variables.

**Part C**
- The objective function maximizes the total skill rating across all selected players. This is appropriate because it ensures that the selected team is not only compliant with the various constraints but also optimizes for overall skill, making the team as competitive as possible. (will add objective function equation in word doc)

**Part D**
- Mathematical notation for the constraint involving player 72 could be expressed as follows, assuming x20 to x24 represent the decision variables for players 20 to 24, and x72 represents player 72 - 
x72 + ∑ i=20 to 24 (xi ≤1)

**Part E**
- Mathematical notation for ensuring at least 30% of the invitations go to guards:
∑i∈Guards (xi ≥0.3×21)

**Part F**
- The optimal objective function value (total skill rating of the selected players) is 300.

**Part G**
- The number of guards (G, G/F) invited to the training camp is 12.

In [45]:
# Part H

from gurobipy import Model, GRB, quicksum
import pandas as pd

# Load the dataset
players_df = pd.read_csv('/Users/mahinbindra/Downloads/BasketballPlayers.csv')  # Update the path to your dataset

# Initialize variables for the model
skills_columns = ['Ball Handling', 'Shooting', 'Rebounding', 'Defense', 'Athletic Ability', 'Toughness', 'Mental Acuity']
guard_flags = [1 if pos in ['G', 'G/F'] else 0 for pos in players_df['Position']]
forward_center_flags = [1 if pos in ['F', 'C', 'F/C'] else 0 for pos in players_df['Position']]

# Iterate to find the smallest number of invitations
for total_invitations in range(21, 0, -1):
    m = Model("team_selection")

    # Decision variables for each player
    x = m.addVars(len(players_df), vtype=GRB.BINARY, name="Player")

    # Position requirements
    m.addConstr(quicksum(x[i] * guard_flags[i] for i in range(len(players_df))) >= 0.3 * total_invitations, "MinGuards")
    m.addConstr(quicksum(x[i] * forward_center_flags[i] for i in range(len(players_df))) >= 0.4 * total_invitations, "MinForwardsCenters")

    # Skill average requirements
    for skill in skills_columns:
        m.addConstr(quicksum(players_df.at[i, skill] * x[i] for i in range(len(players_df))) >= 2.05 * total_invitations, f"Avg{skill}")

    # Specific invitation and exclusion rules as defined in the problem statement
    # Adjust these constraints as needed based on your problem statement

    # Total invitations constraint
    m.addConstr(quicksum(x[i] for i in range(len(players_df))) == total_invitations, "TotalInvitations")

    # Dummy objective, since we're focusing on feasibility
    m.setObjective(quicksum(x[i] for i in range(len(players_df))), GRB.MAXIMIZE)

    # Optimize the model
    m.optimize()

    if m.status == GRB.INFEASIBLE:
        smallest_feasible_invitations = total_invitations + 1
        break

# Print the result
if 'smallest_feasible_invitations' in locals():
    print(f"The smallest number of training camp invitations before yielding an infeasible solution is: {smallest_feasible_invitations}")
else:
    print("The model was feasible with all tested numbers of invitations down to 1.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 150 columns and 1350 nonzeros
Model fingerprint: 0x85fc7b21
Variable types: 0 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e+00, 4e+01]
Presolve removed 0 rows and 5 columns
Presolve time: 0.00s
Presolved: 10 rows, 145 columns, 976 nonzeros
Variable types: 0 continuous, 145 integer (140 binary)
Found heuristic solution: objective 21.0000000

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.02 seconds (0.01 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 21 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.100000000000e+01, best bou

**Part I**
- The main challenge in excluding players with a total score of 12 or under from being invited to the training camp involves calculating the total score for each player based on the given skill ratings and then applying a constraint that only allows the selection of players whose total score exceeds this threshold.

He said not to implement but just in case (chatgpt answer only)

Calculate the total score for each player
players_df['TotalScore'] = players_df[skills_columns].sum(axis=1)

Adding the constraint to the model
for i in range(len(players_df)):
    if players_df.at[i, 'TotalScore'] <= 12:
        m.addConstr(x[i] == 0, f"Exclude_Player_{i}")

**Part J**
- A potential problem could be the lack of consideration for team dynamics or specific game strategies that require more nuanced selection than just skill ratings and positional requirements. For instance, the approach might overlook the importance of player chemistry, leadership qualities, or specific roles that don't directly correlate with the evaluated skills but are crucial for team success.